In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, auc, roc_curve
from sklearn.preprocessing import RobustScaler
import time
import pickle
import os

import warnings
warnings.filterwarnings('ignore')

In [4]:
#reading csv file for the filtered data
columns = ["AccelX(g)_leg","AccelY(g)_leg","AccelZ(g)_leg","GyroX(¬∞/s)_leg","GyroY(¬∞/s)_leg","GyroZ(¬∞/s)_leg","AccelX(g)_hand","AccelY(g)_hand","AccelZ(g)_hand","GyroX(¬∞/s)_hand","GyroY(¬∞/s)_hand","GyroZ(¬∞/s)_hand","AccelX(g)_chest","AccelY(g)_chest","AccelZ(g)_chest","Sample(V)","Activity_Label"]
df = pd.read_csv("resampled_data.csv", header=0, names = columns)

In [7]:
df

,AccelX(g)_leg,AccelY(g)_leg,AccelZ(g)_leg,GyroX(¬∞/s)_leg,GyroY(¬∞/s)_leg,GyroZ(¬∞/s)_leg,AccelX(g)_hand,AccelY(g)_hand,AccelZ(g)_hand,GyroX(¬∞/s)_hand,GyroY(¬∞/s)_hand,GyroZ(¬∞/s)_hand,AccelX(g)_chest,AccelY(g)_chest,AccelZ(g)_chest,Sample(V),Activity_Label
0,-0.208821,-0.681165,0.039955,-0.013333,-0.017493,0.014035,-0.448634,0.200167,0.301584,0.499472,-0.566978,-0.257143,0.472993,0.262520,0.308038,-0.992401,5
1,-0.203130,-0.692338,0.038950,-0.020952,-0.033528,0.014035,-0.445986,0.197873,0.270672,0.440338,-0.561786,-0.259627,0.472993,0.262520,0.308038,-0.992401,5
2,-0.208505,-0.692338,0.042845,-0.028571,-0.042274,0.014035,-0.447786,0.194329,0.238988,0.387540,-0.541018,-0.263354,0.443796,0.244750,0.294567,-0.993414,5
3,-0.206766,-0.679968,0.048624,-0.047619,-0.051020,0.010526,-0.450222,0.190575,0.223725,0.352693,-0.522326,-0.259627,0.443796,0.244750,0.294567,-0.993414,5
4,-0.201865,-0.683160,0.052394,-0.055238,-0.061224,0.017544,-0.449269,0.188699,0.211360,0.336853,-0.504673,-0.250932,0.456934,0.247173,0.318815,-0.990881,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4215910,-1.511937,3.312642,-1.006530,0.662742,-2.441753,-3.630761,-0.219334,-1.154465,-0.880284,-3.078120,3.492286,-4.110654,-0.202920,-1.041195,0.096991,-0.668186,32
4215911,1.219336,-5.738015,1.723957,2.444129,6.522297,15.025655,0.883167,0.415432,-0.192347,0.626031,-0.088331,0.825439,-1.209079,3.310034,0.084292,0.077292,32
4215912,-0.146277,0.844397,-0.562251,3.803145,-1.560415,2.766917,-0.577676,-1.341742,0.476787,1.312728,0.236283,-2.561879,-0.870567,-0.749434,0.194933,0.081014,32
4215913,0.167520,1.451120,-1.219602,2.859403,-6.769342,-4.752712,-0.431536,-1.295434,0.302757,0.233237,-4.445047,1.640737,-0.042390,-0.585679,-0.298924,-0.320796,32


In [8]:
PredictorCol=['AccelX(g)_leg', 'AccelY(g)_leg', 'AccelZ(g)_leg', 'GyroX(¬∞/s)_leg',
       'GyroY(¬∞/s)_leg', 'GyroZ(¬∞/s)_leg', 'AccelX(g)_hand',
       'AccelY(g)_hand', 'AccelZ(g)_hand', 'GyroX(¬∞/s)_hand',
       'GyroY(¬∞/s)_hand', 'GyroZ(¬∞/s)_hand', 'AccelX(g)_chest',
       'AccelY(g)_chest', 'AccelZ(g)_chest', 'Sample(V)']
TargetCol= 'Activity_Label'

X = df[PredictorCol].values
y = df[TargetCol].values

In [9]:
X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X, y, test_size=0.30, 
                                                    random_state=15, 
                                                    stratify = None)

In [11]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50,criterion='gini')
print(clf)
RF=clf.fit(X_train_sm,y_train_sm)
prediction=RF.predict(X_test_sm)
from sklearn import metrics
print(metrics.classification_report(y_test_sm, prediction))
print(metrics.confusion_matrix(y_test_sm, prediction))

RandomForestClassifier(n_estimators=50)
              precision    recall  f1-score   support

           0       0.93      0.96      0.94     38070
           1       0.94      0.95      0.95     38260
           2       0.94      0.96      0.95     38231
           3       0.94      0.96      0.95     38120
           4       0.88      0.94      0.91     38549
           5       0.96      0.96      0.96     38409
           6       0.97      0.98      0.97     38527
           7       0.72      0.62      0.67     38313
           8       0.92      0.94      0.93     38308
           9       1.00      1.00      1.00     38342
          10       1.00      1.00      1.00     38312
          11       0.89      0.88      0.89     38356
          12       0.92      0.94      0.93     38300
          13       0.93      0.96      0.94     38952
          14       0.89      0.88      0.88     38371
          15       1.00      1.00      1.00     38285
          16       0.97      0.96      0.

In [12]:
import pickle

# Save the model to a file
with open("rf.pkl", "wb") as file:
    pickle.dump(RF, file)